## Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', None)

### Applying K means (unsupervised ML ) to some basic data to explain the concept

#### Code along 

kudos to geeks for geeks tutorial from AlindGupta - the ironhack materials are a little heavy going so we are reusing this resource to run through a visual explanation of what KNN/Kmeans is about, together in class 


In [ ]:
# import additional libraries



In [ ]:
# create data array



In [ ]:
# visualise the data 



### Explanation 

visualising the data tells us that 3 clusters looks right but visualising alone is not the full story 

- distortion = the avg of the squared distances from the cluster centres of the respective cluster
- inertia = sum of squared distance of samples to their closest cluster centre

In [ ]:
# build the cluster model , calculate distortion and inertia 



In [ ]:
# tabulate results - distortion



In [ ]:
# visualise results - distortion



In [ ]:
# tabulate results -  inertia



In [ ]:
# visualise results - inertia



### What have you learnt about the application of K-means to cluster data ? 

Your notes here : 

-
-
- 

K-means is an unsupervised learning algorithm for clustering problems, whereas KNN is a supervised learning algorithm for classification and regression problems 

Although both involve similar statistical techniques, you can use KNN to predict by learning from past data, to classify similar data points based on shared characteristics


## Loading data

In [ ]:
#sticking with the healthcare for all dataset 
data = pd.read_csv('lesson_4.05_data.csv') # this file is in files_for_lesson_and_activities folder
data.head()

In [ ]:
data.info()

## Splitting the dataset between the categorical and numerical columns

In [ ]:
categoricals = data.select_dtypes(np.object)
numericals = data.select_dtypes(np.number)


print("Dataframe of numerical columns: ")
print()

display(numericals)

print("Dataframe of categorical columns: ")
print()

display(categoricals)

In [ ]:
#Lets apply scaling to the numerical columns except the final column
#which will be our y variable, average gift

from sklearn.preprocessing import StandardScaler

transformer = StandardScaler().fit(numericals.iloc[:,:-1])
scaled_numericals = transformer.transform(numericals.iloc[:,:-1])

Getting a dataframe with the result from the standard scaler which is a numpy array.

As the numpy arrays doesn't have column names, we pick up the column names from the numerical (original) dataframe.

In [ ]:
pdscaled_numericals= pd.DataFrame(scaled_numericals,columns=numericals.columns.tolist()[:-1])

In [ ]:
pdscaled_numericals.head()

## Dummifying categorical columns

In [ ]:
cat_encoded = pd.get_dummies(categoricals, drop_first=True)
cat_encoded.head()

## Alternative way to do it with OHE: However, you will have to load the corresponding libraries of sklearn.
#encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(categoricals)
#encoded = encoder.transform(categoricals).toarray()

## Putting everything together

In [ ]:
full = pd.concat([pdscaled_numericals.iloc[:,:-1],cat_encoded,numericals.iloc[:,-1]],axis=1)
print("The dataset size is: ", full.shape)
print()
full

In [ ]:
full.columns.tolist()

In [ ]:
full_x = pd.DataFrame(full,columns=full.columns.tolist()[:-1])
full_x.head()

In [ ]:
#define X and y. 
#X is all the features, scaled numericals and encoded categoricals which are our independent variables
#y is the variable we seek to predict 

X = full_x
y = full['AVGGIFT']

## Splitting data intro train and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

## Training the K-NN model

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn_model = KNeighborsRegressor(n_neighbors=4) # The keyword "n_neighbors" is what sets the K.
knn_model.fit(X_train, y_train)

## Getting our predictions

In [ ]:
knn_predictions = knn_model.predict(X_test)
# to get some predictions for y we use the X_test set. 
# we will later compare the predictions from X_test to real y_test 

## Getting the error metrics of our K-NN model

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

MAE = mean_absolute_error(y_test,knn_predictions)
MSE = mean_squared_error(y_test,knn_predictions)
RMSE = np.sqrt(MSE)
R2 = r2_score(y_test,knn_predictions)

print("The mean absolute error of the model in the test set is: %6.2f" % (MAE))
print("The mean squared error of the model in the test set is: %6.2f" % (MSE))
print("The root mean squared error of the model in the test set is: %6.2f" % (RMSE))
print("The R2 of the model in the test set is: %4.2f" % (R2))

# Activity

Train a linear model and compare the performance of both models in the test set.


In [ ]:
#HINT : from sklearn import linear_model



In [ ]:
#HINT : lm_predictions = lm_model.predict(X_test)


In [ ]:
#HINT : use the same metrics summary cell as above 


# Activity

If you think a little bit about it, the number of neighbors might be very important for our results, but will it be the only parameter that matters? Go to the documentation and check the parameters and the values they can take, pick the one you think is more relevant and change its value in the model. 

Hint: If K (number of neighbors) is the most important one, maybe we could measure the way these K instances affect our prediction

In [ ]:
uniform_model = KNeighborsRegressor(n_neighbors=9)
uniform_model.fit(X_train, y_train)
uniform_model.score(X_test, y_test)

In [ ]:
# example using parameter distance 
distance_model = KNeighborsRegressor(n_neighbors=9, weights = "distance")
distance_model.fit(X_train, y_train)
distance_model.score(X_test, y_test)

# visualise KNN

Let's visualize how KNN actually works. First of all install the mlxtend library and create a dataframe containing the two most relevant numerical variables and the target, in that order. Once you have done it sample it with n = 100, introduce that sample into this function with an arbitrary k:

In [ ]:
#to do this aou will need to close and halt your notebook & install mlxtend library

#I had to first install pip (conda install pip) 
# then run pip install mlxtend

In [ ]:
from mlxtend.plotting import plot_decision_regions

In [ ]:
def knn_comparison(data, k):
    x = data.iloc[:, 0:2].values
    y = data.iloc[:, -1].astype(int).values
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(x, y)

    plt.figure(figsize=(16,12))
    plot_decision_regions(x, y, clf=knn)
    plt.title("Knn with K="+ str(k), fontsize = 18)
    plt.show()

In [ ]:
new = pd.concat([X,y],axis=1)
new = new[['HV1','IC1','AVGGIFT']].sample(n=100,random_state=100)
new.head()

In [ ]:
# run the plot - be prepared this will look like crazy modern art
knn_comparison(new,1)

### Activity - take time to look at the chart and reflect on the data
What can you see in the plot? Try re running the cell to plot the data with a higher or lower k (1,2,4,9,15,30)



#### explanation 
The lower the number of k the more over-fitted it will be. We can see that with k = 1, the boundaries are very clear and as we increase k the plots start turning very messy until the last two plots, where it is oversimplified.

# Metrics Review (Linear Regression):

From all the regression metrics we have seen, which one(s) do you think you will use in most cases?


We have seen R2, MSE, RMSE and MAE. Of course, there is not a magic solution for which you should always use it, but there are some details worth knowing:

*  R2 is scaled, which means that it is independent of the data. This one would be the one to go with if we don't know a lot about the data and general information about our model. However, it can be misleading, as it is supposed to be between 0 and 1 but sometimes it is not (you can read about it here. In fact, R2 is a biased estimator (more information here.

*  MAE would be the median of the regression metrics as what it measures is the sum of distances between predicted and real values (errors), and that won't give a special treat to really bad predictions, so if that's what we want this metric should do great.

*  MSE - It is the mean of the squared distance of the errors, which will weight the bad predictions.

*  RMSE - Root MSE, essentially it is the same but it is easier to understand within the data context

### Activity 

Focusing on either the linear or knn model it can be useful to calculate the R2 adjusted- read through the guide below and run the last cell to calculate the metric. Hint: you will need to refer to r2, so first you should set the parameter (hard code it) from one of the earlier model summary cells 

- r_squared = 


#### R2 and R2-adjusted 
R2 shows how well terms (data points) fit a curve or line. Adjusted R2 also indicates how well terms fit a curve or line, but adjusts for the number of terms in a model. If you add more and more useless variables to a model, adjusted r-squared will decrease. If you add more useful variables, adjusted r-squared will increase.
Adjusted R2 will always be less than or equal to R2.

--------

#### The adjusted R-squared is calculated as:

Adjusted R2 = 1 – [(1-R2)*(n-1)/(n-k-1)]

where:

- R2: The R2 of the model
- n: The number of observations
- k: The number of predictor variables

--------

Both R2 and the adjusted R2 give you an idea of how many data points fall within the line of the regression equation. However, there is one main difference between R2 and the adjusted R2: R2 assumes that every single variable explains the variation in the dependent variable. The adjusted R2 tells you the percentage of variation explained by only the independent variables that actually affect the dependent variable.

NOTE : You only need R2 when working with samples. In other words, R2 isn’t necessary when you have data from an entire population.

In [ ]:
#R2adj can be computed with python as:
    
score_adj = 1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)